参考：https://towardsdatascience.com/conditional-text-generation-by-fine-tuning-gpt-2-11c1a9fc639d

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/huggingface/transformers  # ソースコードから直接transformersをインストール
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wkeirgst
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-wkeirgst
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 5.2 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
  Created wheel for transformers: filename=transformers-4.21.0.dev0-py3-none-any.whl size=4584376 sha256=464b11172cd13326ea66d1ab5398f073a2b4b53e6a4a5bad9ee3e1866197e375
  Stored in directory: /tmp/pip-ephem-wheel-cache-ykw5i_pg/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninst

In [3]:
!git clone https://github.com/huggingface/transformers

# run_clm.pyを使う
!ls ./transformers/examples/pytorch/language-modeling/
# README.md   run_clm_no_trainer.py  run_mlm_no_trainer.py  run_plm.py
# requirements.txt  run_clm.py       run_mlm.py

Cloning into 'transformers'...
remote: Enumerating objects: 100221, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 100221 (delta 61), reused 108 (delta 45), pack-reused 100060
Receiving objects: 100% (100221/100221), 93.69 MiB | 26.78 MiB/s, done.
Resolving deltas: 100% (73860/73860), done.
README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [4]:
MODEL = "gpt2"
OUTPUT_DIR = "drive/MyDrive/colab/output_gpt2/"
DATA_PATH = "drive/MyDrive/colab/data/rocstories.txt"
DATA_JOINED_PATH = "drive/MyDrive/colab/data/data_joined.txt"
SPECIAL_TOKENS = { "bos_token": "<|BOS|>",
                   "eos_token": "<|EOS|>",
                   "unk_token": "<|UNK|>",                    
                   "pad_token": "<|PAD|>",
                   "sep_token": "<|SEP|>"}

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
# AutoModelForPreTraining
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.add_special_tokens(SPECIAL_TOKENS)
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

config = AutoConfig.from_pretrained(MODEL,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    output_hidden_states=False)
model = AutoModelForCausalLM.from_pretrained(MODEL, config=config)
#Special tokens added, model needs to be resized accordingly
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Embedding(50262, 768)

In [6]:
import random

data_list = []
sent = ""

with open(DATA_PATH, 'r', encoding="shift-jis") as f:
  data_list = f.readlines()

with open(DATA_JOINED_PATH, 'w', encoding="utf-8") as f:  
  for data in data_list:
    pieces = data.split()
    sent = SPECIAL_TOKENS['bos_token'] + pieces[random.randrange(len(pieces))] + SPECIAL_TOKENS['sep_token'] + " ".join(pieces).replace("\n", "") + SPECIAL_TOKENS['eos_token']
    f.write(sent + "\n")

In [7]:
print(sent)

<|BOS|>me<|SEP|>My parents gave me enough money to buy the console.<|EOS|>


In [8]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=gpt2 \
    --train_file=drive/MyDrive/colab/data/data_joined.txt \
    --validation_file=drive/MyDrive/colab/data/data_joined.txt \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --do_train \
    --do_eval \
    --output_dir=drive/MyDrive/colab/output_gpt2/ \
    --num_train_epochs=5 \
    --save_total_limit=3 \
    --save_steps=10000 \
    --overwrite_output_dir

"""
    --model_name_or_path=gpt2 \
    --train_file=drive/MyDrive/colab/data/data_joined.txt \
    --validation_file=drive/MyDrive/colab/data/data_joined.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=5 \
    --save_total_limit=3 \
    --save_steps=10000 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=drive/MyDrive/colab/output_gpt2/ \
    --use_fast_tokenizer=False \
    --overwrite_output_dir
"""

07/07/2022 14:25:26 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/07/2022 14:25:26 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_

'\n    --model_name_or_path=gpt2     --train_file=drive/MyDrive/colab/data/data_joined.txt     --validation_file=drive/MyDrive/colab/data/data_joined.txt     --do_train     --do_eval     --num_train_epochs=5     --save_total_limit=3     --save_steps=10000     --per_device_train_batch_size=1     --per_device_eval_batch_size=1     --output_dir=drive/MyDrive/colab/output_gpt2/     --use_fast_tokenizer=False     --overwrite_output_dir\n'

In [9]:
model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.do_lower_case = True
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [24]:
def conditional_generation(keyword, num_gen=20):
    input_text = SPECIAL_TOKENS['bos_token']+keyword+SPECIAL_TOKENS['sep_token']
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    out = model.generate(input_ids, do_sample=True, top_p=0.95, top_k=40, num_return_sequences=num_gen, max_length=50, bad_words_ids=[[1], [3]])
    print("keyword: "+keyword)
    print("generated:")
    for sent in tokenizer.batch_decode(out):
      sent = sent.split(SPECIAL_TOKENS['sep_token'])[1].split(SPECIAL_TOKENS['eos_token'])[0]
      print(sent)

In [25]:
conditional_generation("happy")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


keyword: happy
generated:
Amber is happy because it is the only day her son is a year old.
Livie wanted a new house.
Joey is happy when everyone is happy.
Neil and Ellen were happy to meet.
Sandra was happy to be back home.
The happy wife said, "I was happy last night.
Ben and I went to the movie together.
We all got to eat delicious pizza today.
Cameron was happy to have the chance to do something he loves.
He was happy to have made a change.
Bob was happy he had gotten some of the candy.
It's been a sad day for Kimmy and her family.
A happy, sad, happy Andrew decided to try out for the soccer team.
The boy felt happy to be happy and happy.
He always had a happy childhood.
He was happy to finally have found a new job.
Joe and Brenda were both happy that they had met.
Nora was nervous about the future at school.
Amy is happy she was able to have a baby boy.
They had a happy birthday with him.
